In [1]:
import logging
import sys
import os
from os import listdir
from os.path import isfile, join, isdir
import numpy as np
import tqdm
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import tensorflow.compat.v1.keras as keras
import pickle
import math
import time
import shutil
import glob
import random as rd

Instructions for updating:
non-resource variables are not supported in the long term


In [2]:
count = 0
countBothWays = 0

onionAddressData = {}

flow_size = 100

negetive_samples = 9
BATCH_SIZE = 128
EPOCH_COUNT = 200
learn_rate=0.0001

config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.Session(config=config)
keras.backend.set_session(session)

datasetScalerTimes = 1000
datasetScalerSizes = 1/1000

CNN_21_41 = {'kernelSize1':[2, 30], 'stride1':[2,1], 'kernelSize2':[4, 10], 'stride2':[4,1]} 

reduce_factor = 16

parameters ={
    'cnn_secondStage_21_41':CNN_21_41,
}
    
baseDatasetPath = '/home/jupyter-atse/datasets/'

metaDataPairsFolders = pickle.load(open(baseDatasetPath + 'metaDataPairsFolders', 'rb'))
trainPairsFolders = pickle.load(open(baseDatasetPath + 'trainPairsFolders', 'rb'))
testPairsFolders = pickle.load(open(baseDatasetPath + 'testPairsFolders', 'rb'))

onionAddressData = metaDataPairsFolders['onionAddressData']

In [3]:
def generateDataset(pairsFoldersInput):

    global negetive_samples, flow_size, onionAddressData, graphTitle

    for onionUrl in onionAddressData:
        onionAddressData[onionUrl]['connectionIndex'] = 0

    index = 0

    l2s=np.zeros((len(pairsFoldersInput)*(negetive_samples+1),8,flow_size,1))
    labels=np.zeros((len(pairsFoldersInput)*(negetive_samples+1),1))
    for pairFolder in pairsFoldersInput:
        
        clientTimesIn = pairFolder['clientFlow']['timesIn']
        clientTimesOut = pairFolder['clientFlow']['timesOut']
        clientSizesIn = pairFolder['clientFlow']['sizesIn']
        clientSizesOut = pairFolder['clientFlow']['sizesOut']

        hsTimesIn = pairFolder['hsFlow']['timesIn']
        hsTimesOut = pairFolder['hsFlow']['timesOut']
        hsSizesIn = pairFolder['hsFlow']['sizesIn']
        hsSizesOut = pairFolder['hsFlow']['sizesOut']


        l2s[index,0,:len(clientTimesIn[:flow_size]),0]=np.array(clientTimesIn[:flow_size])*datasetScalerTimes
        l2s[index,1,:len(hsTimesOut[:flow_size]),0]=np.array(hsTimesOut[:flow_size])*datasetScalerTimes
        l2s[index,2,:len(hsTimesIn[:flow_size]),0]=np.array(hsTimesIn[:flow_size])*datasetScalerTimes
        l2s[index,3,:len(clientTimesOut[:flow_size]),0]=np.array(clientTimesOut[:flow_size])*datasetScalerTimes

        l2s[index,4,:len(clientSizesIn[:flow_size]),0]=np.array(clientSizesIn[:flow_size])*datasetScalerSizes
        l2s[index,5,:len(hsSizesOut[:flow_size]),0]=np.array(hsSizesOut[:flow_size])*datasetScalerSizes
        l2s[index,6,:len(hsSizesIn[:flow_size]),0]=np.array(hsSizesIn[:flow_size])*datasetScalerSizes
        l2s[index,7,:len(clientSizesOut[:flow_size]),0]=np.array(clientSizesOut[:flow_size])*datasetScalerSizes

        labels[index, 0] = 1

        index += 1

        random = list(range(len(pairsFoldersInput)))
        np.random.shuffle(random)
        negetive_samples_current = 0
        for negetive_pair_index in random:

            #skip if is the original correlated pair
            if pairsFoldersInput[negetive_pair_index]['hsFolder'] == pairFolder['hsFolder']:
                continue

            hsTimesIn = pairsFoldersInput[negetive_pair_index]['hsFlow']['timesIn']
            hsTimesOut = pairsFoldersInput[negetive_pair_index]['hsFlow']['timesOut']
            hsSizesIn = pairsFoldersInput[negetive_pair_index]['hsFlow']['sizesIn']
            hsSizesOut = pairsFoldersInput[negetive_pair_index]['hsFlow']['sizesOut']

            
            l2s[index,0,:len(clientTimesIn[:flow_size]),0]=np.array(clientTimesIn[:flow_size])*datasetScalerTimes
            l2s[index,1,:len(hsTimesOut[:flow_size]),0]=np.array(hsTimesOut[:flow_size])*datasetScalerTimes
            l2s[index,2,:len(hsTimesIn[:flow_size]),0]=np.array(hsTimesIn[:flow_size])*datasetScalerTimes
            l2s[index,3,:len(clientTimesOut[:flow_size]),0]=np.array(clientTimesOut[:flow_size])*datasetScalerTimes

            l2s[index,4,:len(clientSizesIn[:flow_size]),0]=np.array(clientSizesIn[:flow_size])*datasetScalerSizes
            l2s[index,5,:len(hsSizesOut[:flow_size]),0]=np.array(hsSizesOut[:flow_size])*datasetScalerSizes
            l2s[index,6,:len(hsSizesIn[:flow_size]),0]=np.array(hsSizesIn[:flow_size])*datasetScalerSizes
            l2s[index,7,:len(clientSizesOut[:flow_size]),0]=np.array(clientSizesOut[:flow_size])*datasetScalerSizes


            labels[index, 0] = 0

            index += 1

            negetive_samples_current += 1
            if negetive_samples_current >= negetive_samples:
                break

        onionAddressData[pairFolder['onionAddress']]['connectionIndex'] += 1

    return l2s, labels

In [4]:
for modelParametersKey in parameters:
    
    #create directory for models
    if not os.path.exists(modelParametersKey):
        os.makedirs(modelParametersKey)


    #model definition
    model = keras.Sequential()

    orig = [2000/reduce_factor, 1000/reduce_factor, 49600/reduce_factor, 3000/reduce_factor, 800/reduce_factor, 100/reduce_factor]
    #dropout_prob = 0.4
    model.add(keras.layers.Conv2D(int(orig[0]), input_shape=[8, flow_size, 1], kernel_size=parameters[modelParametersKey]['kernelSize1'], strides=parameters[modelParametersKey]['stride1'], padding='VALID', kernel_initializer=keras.initializers.RandomNormal(mean=0.0, stddev=0.01, seed=None)))
    model.add(keras.layers.Activation('relu'))
    model.add(keras.layers.MaxPool2D(pool_size=(1, 5), strides=(1, 1), padding='VALID'))

    model.add(keras.layers.Conv2D(int(orig[1]), kernel_size=parameters[modelParametersKey]['kernelSize2'], strides=parameters[modelParametersKey]['stride2'], padding='VALID', kernel_initializer=keras.initializers.RandomNormal(mean=0.0, stddev=0.01, seed=None)))
    model.add(keras.layers.Activation('relu'))
    model.add(keras.layers.MaxPool2D(pool_size=(1, 5), strides=(1, 1), padding='VALID')) 

    model.add(keras.layers.Flatten())
    model.add(keras.layers.Dense(int(orig[3]), kernel_initializer=keras.initializers.RandomNormal(mean=0.0, stddev=0.01, seed=None)))
    model.add(keras.layers.Activation('relu'))
    #model.add(keras.layers.Dropout(dropout_prob))
    model.add(keras.layers.Dense(int(orig[4]), kernel_initializer=keras.initializers.RandomNormal(mean=0.0, stddev=0.01, seed=None)))
    model.add(keras.layers.Activation('relu'))
    #model.add(keras.layers.Dropout(dropout_prob))
    model.add(keras.layers.Dense(int(orig[5]), kernel_initializer=keras.initializers.RandomNormal(mean=0.0, stddev=0.01, seed=None)))
    model.add(keras.layers.Activation('relu'))
    #model.add(keras.layers.Dropout(dropout_prob))
    model.add(keras.layers.Dense(1))

    def customLoss(y_true,y_pred):
        return tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=y_pred,labels=y_true),name='loss_sigmoid')



    model.compile(loss=customLoss, optimizer=tf.keras.optimizers.Adam(learning_rate=learn_rate), metrics=['accuracy'])
    # custom function
    def sigmoid(x):
        if x < -150:
            x = -150
        if x > 150:
            x = 150
        return 1 / (1 + math.exp(-x))

    # define vectorized sigmoid
    sigmoid_v = np.vectorize(sigmoid)



    losses = [10, 10]
    saveNames = ['', '']
    precisionBest = 0
    saveNamePrecisionBest = ''
    f1_05 = 0
    saveNameF1_05 = ''

    correlatedPerRound = 10


    tpBest = 0
    tp = { 0.01 : 0, 0.05 : 0, 0.1 : 0, 0.2 : 0, 0.3 : 0, 0.4 : 0, 0.5 : 0, 0.6 : 0, 0.7 : 0, 0.8 : 0, 0.9 : 0, 0.95 : 0, 0.99 : 0}
    fpBest = 0
    fp = { 0.01 : 0, 0.05 : 0, 0.1 : 0, 0.2 : 0, 0.3 : 0, 0.4 : 0, 0.5 : 0, 0.6 : 0, 0.7 : 0, 0.8 : 0, 0.9 : 0, 0.95 : 0, 0.99 : 0}
    tn = { 0.01 : 0, 0.05 : 0, 0.1 : 0, 0.2 : 0, 0.3 : 0, 0.4 : 0, 0.5 : 0, 0.6 : 0, 0.7 : 0, 0.8 : 0, 0.9 : 0, 0.95 : 0, 0.99 : 0}
    fn = { 0.01 : 0, 0.05 : 0, 0.1 : 0, 0.2 : 0, 0.3 : 0, 0.4 : 0, 0.5 : 0, 0.6 : 0, 0.7 : 0, 0.8 : 0, 0.9 : 0, 0.95 : 0, 0.99 : 0}

    start = time.time()
    try:

        for epoch in tqdm.tqdm(range(EPOCH_COUNT)):
            l2s, labels = generateDataset(trainPairsFolders)
            rr= list(range(len(l2s)))
            np.random.shuffle(rr)
            l2s=l2s[rr]
            labels=labels[rr]
            
            history = model.fit(l2s, labels, batch_size=BATCH_SIZE, verbose=0)
            loss = history.history['loss'][-1]
            print("Epoch:", epoch, "loss:", loss)
            

            path = 'epoch' + str(epoch) + '_loss' + str(loss) + '/cp.ckpt'
            model.save_weights(modelParametersKey + '/' + 'last_' + path)
            for filename in glob.glob(modelParametersKey + '/' + 'last_' + 'epoch' + str(epoch-1) + '*'):
                shutil.rmtree(filename)




            #save model based on precision
            if epoch % 3 == 0 and epoch > 0:
                for onionUrl in onionAddressData:
                    onionAddressData[onionUrl]['connectionIndex'] = 0

                for i in tqdm.tqdm(range(len(testPairsFolders)//correlatedPerRound-1)):
                    #print(i)
                    l2s_test=np.zeros(((negetive_samples+1)*correlatedPerRound,8,flow_size,1))
                    labels_test=np.zeros(((negetive_samples+1)*correlatedPerRound))

                    index = 0

                    for pairFolder in testPairsFolders[i*correlatedPerRound:(i+1)*correlatedPerRound]:

                        clientTimesIn = pairFolder['clientFlow']['timesIn']
                        clientTimesOut = pairFolder['clientFlow']['timesOut']
                        clientSizesIn = pairFolder['clientFlow']['sizesIn']
                        clientSizesOut = pairFolder['clientFlow']['sizesOut']

                        hsTimesIn = pairFolder['hsFlow']['timesIn']
                        hsTimesOut = pairFolder['hsFlow']['timesOut']
                        hsSizesIn = pairFolder['hsFlow']['sizesIn']
                        hsSizesOut = pairFolder['hsFlow']['sizesOut']                  

                        l2s_test[index,0,:len(clientTimesIn[:flow_size]),0]=np.array(clientTimesIn[:flow_size])*datasetScalerTimes
                        l2s_test[index,1,:len(hsTimesOut[:flow_size]),0]=np.array(hsTimesOut[:flow_size])*datasetScalerTimes
                        l2s_test[index,2,:len(hsTimesIn[:flow_size]),0]=np.array(hsTimesIn[:flow_size])*datasetScalerTimes
                        l2s_test[index,3,:len(clientTimesOut[:flow_size]),0]=np.array(clientTimesOut[:flow_size])*datasetScalerTimes

                        l2s_test[index,4,:len(clientSizesIn[:flow_size]),0]=np.array(clientSizesIn[:flow_size])*datasetScalerSizes
                        l2s_test[index,5,:len(hsSizesOut[:flow_size]),0]=np.array(hsSizesOut[:flow_size])*datasetScalerSizes
                        l2s_test[index,6,:len(hsSizesIn[:flow_size]),0]=np.array(hsSizesIn[:flow_size])*datasetScalerSizes
                        l2s_test[index,7,:len(clientSizesOut[:flow_size]),0]=np.array(clientSizesOut[:flow_size])*datasetScalerSizes

                        labels_test[index] = 1

                        index += 1

                        random = list(range(len(testPairsFolders)))
                        np.random.shuffle(random)
                        negetive_samples_current = 0
                        for negetive_pair_index in random:

                            #if is the corresponding 
                            if testPairsFolders[negetive_pair_index]['hsFolder'] == pairFolder['hsFolder']:
                                continue


                            hsTimesIn = testPairsFolders[negetive_pair_index]['hsFlow']['timesIn']
                            hsTimesOut = testPairsFolders[negetive_pair_index]['hsFlow']['timesOut']
                            hsSizesIn = testPairsFolders[negetive_pair_index]['hsFlow']['sizesIn']
                            hsSizesOut = testPairsFolders[negetive_pair_index]['hsFlow']['sizesOut']   


                            l2s_test[index,0,:len(clientTimesIn[:flow_size]),0]=np.array(clientTimesIn[:flow_size])*datasetScalerTimes
                            l2s_test[index,1,:len(hsTimesOut[:flow_size]),0]=np.array(hsTimesOut[:flow_size])*datasetScalerTimes
                            l2s_test[index,2,:len(hsTimesIn[:flow_size]),0]=np.array(hsTimesIn[:flow_size])*datasetScalerTimes
                            l2s_test[index,3,:len(clientTimesOut[:flow_size]),0]=np.array(clientTimesOut[:flow_size])*datasetScalerTimes

                            l2s_test[index,4,:len(clientSizesIn[:flow_size]),0]=np.array(clientSizesIn[:flow_size])*datasetScalerSizes
                            l2s_test[index,5,:len(hsSizesOut[:flow_size]),0]=np.array(hsSizesOut[:flow_size])*datasetScalerSizes
                            l2s_test[index,6,:len(hsSizesIn[:flow_size]),0]=np.array(hsSizesIn[:flow_size])*datasetScalerSizes
                            l2s_test[index,7,:len(clientSizesOut[:flow_size]),0]=np.array(clientSizesOut[:flow_size])*datasetScalerSizes


                            labels_test[index] = 0

                            index += 1

                            negetive_samples_current += 1
                            if negetive_samples_current >= negetive_samples:
                                break

                    #ignore batch size and use batches of negetive_samples+1
                    prediction = model.predict(l2s_test)
                    break
                    results = sigmoid_v(prediction).tolist()
                    #'''




    except KeyboardInterrupt:
        pass

    #model.save_weights(checkpoint_path)
    #print("Saved")

    #end = time.time()
    #print('Training time(s): ' + str(end - start))
    
    this_execution = np.zeros(10)
    l3s, labels3 = generateDataset(testPairsFolders)
    for j in range(10):
        start_test = time.time()
        test_loss, test_accuracy = model.evaluate(l3s,labels3, batch_size=BATCH_SIZE, verbose=1)
        end_test = time.time()
        this_execution[j] = end_test - start_test
    execution_time = np.min(this_execution)
    precision_value = test_accuracy


Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


  0%|          | 1/200 [00:04<15:19,  4.62s/it]

Epoch: 0 loss: 0.39411216156496


  1%|          | 2/200 [00:06<09:17,  2.82s/it]

Epoch: 1 loss: 0.3497788623705165


...

 99%|█████████▉| 198/200 [05:03<00:03,  1.51s/it]

Epoch: 197 loss: 0.022043439215632305
Epoch: 198 loss: 0.05712147181944879



100%|██████████| 200/200 [05:06<00:00,  1.53s/it]

Epoch: 199 loss: 0.027203624910788735



/opt/tljh/user/lib/python3.9/site-packages/keras/engine/training_v1.py:2057: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates = self.state_updates


In [5]:
print('Runtime = ', execution_time)
print('Accuracy = ', precision_value)

Runtime =  0.21114492416381836
Accuracy =  0.91821563
